降维就是一种对高维度特征数据预处理方法。降维是将高维度的数据保留下最重要的一些特征，去除噪声和不重要的特征，从而实现提升数据处理速度的目的。在实际的生产和应用中，降维在一定的信息损失范围内，可以为我们节省大量的时间和成本。降维也成为应用非常广泛的数据预处理方法。

PCA 就是这样的降维方法

PCA(Principal Component Analysis)，即主成分分析方法，是一种使用最广泛的数据降维算法。PCA的主要思想是将n维特征映射到k维上，这k维是全新的正交特征也被称为主成分，是在原有n维特征的基础上重新构造出来的k维特征。PCA的工作就是从原始的空间中顺序地找一组相互正交的坐标轴，新的坐标轴的选择与数据本身是密切相关的。其中，第一个新坐标轴选择是原始数据中方差最大的方向，第二个新坐标轴选取是与第一个坐标轴正交的平面中使得方差最大的，第三个轴是与第1,2个轴正交的平面中方差最大的。依次类推，可以得到n个这样的坐标轴。通过这种方式获得的新的坐标轴，我们发现，大部分方差都包含在前面k个坐标轴中，后面的坐标轴所含的方差几乎为0。于是，我们可以忽略余下的坐标轴，只保留前面k个含有绝大部分方差的坐标轴。事实上，这相当于只保留包含绝大部分方差的维度特征，而忽略包含方差几乎为0的特征维度，实现对数据特征的降维处理。

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

data = [
    # [最大速度, 飞行半径, 最大负载, 费用, 可靠性, 灵敏度]
    [2.0, 1500, 20000, 5500000, 0.5, 1.0],  # A1
    [2.5, 2700, 18000, 6500000, 0.3, 0.5],  # A2
    [1.8, 2000, 21000, 4500000, 0.7, 0.7],  # A3
    [2.2, 1800, 20000, 5000000, 0.5, 0.5],  # A4
]

aircraft_names = ["A1", "A2", "A3", "A4"]
indicator_names = ["最大速度", "飞行半径", "最大负载", "费用", "可靠性", "灵敏度"]

df = pd.DataFrame(data=data, columns=indicator_names, index=aircraft_names)

X = np.array(data)

X = StandardScaler().fit_transform(X)

pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

print(X_reduced)  # 降维后的结果

[[ 1.0922184   1.7104114 ]
 [-3.4805083  -0.08995552]
 [ 2.25756044 -0.92597504]
 [ 0.13072946 -0.69448083]]


In [12]:
pca.explained_variance_ratio_ # 各主成分的方差百分比

array([0.76752285, 0.17805552])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 设置中文字体
plt.rcParams["font.sans-serif"] = ["SimHei", "DejaVu Sans"]
plt.rcParams["axes.unicode_minus"] = False

print("=" * 60)
print("【PCA主成分分析详细结果】")
print("=" * 60)

# 1. 各主成分的方差解释比例
print(
    f"第一主成分解释方差比例: {pca.explained_variance_ratio_[0]:.4f} ({pca.explained_variance_ratio_[0]*100:.2f}%)"
)
print(
    f"第二主成分解释方差比例: {pca.explained_variance_ratio_[1]:.4f} ({pca.explained_variance_ratio_[1]*100:.2f}%)"
)
print(
    f"累计解释方差比例: {pca.explained_variance_ratio_.sum():.4f} ({pca.explained_variance_ratio_.sum()*100:.2f}%)"
)

# 2. 主成分载荷矩阵(各原始指标对主成分的贡献)
print("\n【主成分载荷矩阵】")
print("-" * 40)
components_df = pd.DataFrame(
    pca.components_.T, columns=["第一主成分", "第二主成分"], index=indicator_names
)
print(components_df.round(4))

# 3. 计算各指标对主成分的贡献率
print("\n【各指标对主成分的贡献率】")
print("-" * 40)
# 贡献率 = (载荷值^2) / 特征值
eigenvalues = pca.explained_variance_
contribution_rates = pd.DataFrame(
    (pca.components_.T**2) / eigenvalues,
    columns=["对PC1贡献率", "对PC2贡献率"],
    index=indicator_names,
)
print(contribution_rates.round(4))

# 4. 原始数据在主成分空间的坐标
print("\n【降维后的主成分得分】")
print("-" * 40)
pc_scores_df = pd.DataFrame(
    X_reduced, columns=["第一主成分得分", "第二主成分得分"], index=aircraft_names
)
print(pc_scores_df.round(4))

# 5. 构建基于PCA的综合评价模型
print("\n【基于PCA的综合评价模型】")
print("-" * 40)

# 使用主成分得分和方差解释比例计算综合得分
weights = pca.explained_variance_ratio_  # 以方差解释比例作为权重
comprehensive_scores = np.dot(X_reduced, weights)

# 标准化到0-1区间
min_score = comprehensive_scores.min()
max_score = comprehensive_scores.max()
normalized_scores = (comprehensive_scores - min_score) / (max_score - min_score)

# 创建评价结果DataFrame
evaluation_results = pd.DataFrame(
    {
        "第一主成分得分": X_reduced[:, 0],
        "第二主成分得分": X_reduced[:, 1],
        "综合得分": comprehensive_scores,
        "标准化得分": normalized_scores,
        "排名": pd.Series(comprehensive_scores).rank(ascending=False).astype(int),
    },
    index=aircraft_names,
)

print(evaluation_results.round(4))
# 7. 生成详细报告
print("\n【PCA评价模型分析报告】")
print("=" * 50)
print(f" 数据概况:")
print(f"   - 原始维度: {X.shape[1]}维")
print(f"   - 降维后维度: 2维")
print(f"   - 信息保留率: {pca.explained_variance_ratio_.sum()*100:.2f}%")

print(f"\n 排名结果:")
ranking = evaluation_results.sort_values("综合得分", ascending=False)
for i, (aircraft, row) in enumerate(ranking.iterrows(), 1):
    print(f"   第{i}名: {aircraft} (得分: {row['综合得分']:.4f})")

print(f"\n 主成分解释:")
print(f"   - 第一主成分(PC1): 解释{pca.explained_variance_ratio_[0]*100:.2f}%的方差")
print(f"     主要反映指标: ", end="")
pc1_important = components_df["第一主成分"].abs().nlargest(3)
print(", ".join([f"{idx}({val:.3f})" for idx, val in pc1_important.items()]))

print(f"   - 第二主成分(PC2): 解释{pca.explained_variance_ratio_[1]*100:.2f}%的方差")
print(f"     主要反映指标: ", end="")
pc2_important = components_df["第二主成分"].abs().nlargest(3)
print(", ".join([f"{idx}({val:.3f})" for idx, val in pc2_important.items()]))

print(f"\n建议:")
best_aircraft = ranking.index[0]
print(f"   - 推荐选择: {best_aircraft}")
print(f"   - 该飞机在综合评价中表现最佳，平衡了各项性能指标")

【PCA主成分分析详细结果】
第一主成分解释方差比例: 0.7675 (76.75%)
第二主成分解释方差比例: 0.1781 (17.81%)
累计解释方差比例: 0.9456 (94.56%)

【主成分载荷矩阵】
----------------------------------------
       第一主成分   第二主成分
最大速度 -0.4546  0.0012
飞行半径 -0.3699 -0.4130
最大负载  0.4592 -0.1602
费用   -0.4260  0.3744
可靠性   0.4405 -0.2767
灵敏度   0.2647  0.7662

【各指标对主成分的贡献率】
----------------------------------------
      对PC1贡献率  对PC2贡献率
最大速度   0.0337   0.0000
飞行半径   0.0223   0.1197
最大负载   0.0343   0.0180
费用     0.0296   0.0984
可靠性    0.0316   0.0537
灵敏度    0.0114   0.4121

【降维后的主成分得分】
----------------------------------------
    第一主成分得分  第二主成分得分
A1   1.0922   1.7104
A2  -3.4805  -0.0900
A3   2.2576  -0.9260
A4   0.1307  -0.6945

【基于PCA的综合评价模型】
----------------------------------------
    第一主成分得分  第二主成分得分    综合得分   标准化得分  排名
A1   1.0922   1.7104  1.1429  0.9001 NaN
A2  -3.4805  -0.0900 -2.6874  0.0000 NaN
A3   2.2576  -0.9260  1.5679  1.0000 NaN
A4   0.1307  -0.6945 -0.0233  0.6261 NaN

【PCA评价模型分析报告】
 数据概况:
   - 原始维度: 6维
   - 降维后维度: 2维
   - 信息保留率: 9